## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,76.48,64,90,7.81,overcast clouds
1,1,Albany,US,42.6001,-73.9662,70.38,78,21,3.22,few clouds
2,2,Longyearbyen,SJ,78.2186,15.6401,33.64,93,75,11.50,light rain
3,3,Rikitea,PF,-23.1203,-134.9692,73.54,79,100,19.17,light rain
4,4,Ushuaia,AR,-54.8000,-68.3000,49.66,53,40,4.61,scattered clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,76.48,64,90,7.81,overcast clouds
5,5,Abu Dhabi,AE,24.4667,54.3667,89.71,74,0,8.05,clear sky
6,6,Maragogi,BR,-9.0122,-35.2225,80.56,73,95,11.43,overcast clouds
8,8,Aljezur,PT,37.3191,-8.8033,75.76,55,72,7.02,broken clouds
10,10,Kapaa,US,22.0752,-159.3190,76.19,80,1,13.80,light rain
14,14,Itarema,BR,-2.9248,-39.9167,88.65,57,48,19.26,scattered clouds
22,22,Hilo,US,19.7297,-155.0900,75.27,90,90,4.61,overcast clouds
23,23,Bandarbeyla,SO,9.4942,50.8122,80.01,80,31,19.53,light rain
24,24,Alice Town,BS,25.7167,-79.3000,82.89,72,44,10.38,scattered clouds
25,25,Kununurra,AU,-15.7667,128.7333,76.98,69,59,0.00,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                215
City                   215
Country                215
Lat                    215
Lng                    215
Max Temp               215
Humidity               215
Cloudiness             215
Wind Speed             215
Current Description    215
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Castro,BR,76.48,overcast clouds,-24.7911,-50.0119,
5,Abu Dhabi,AE,89.71,clear sky,24.4667,54.3667,
6,Maragogi,BR,80.56,overcast clouds,-9.0122,-35.2225,
8,Aljezur,PT,75.76,broken clouds,37.3191,-8.8033,
10,Kapaa,US,76.19,light rain,22.0752,-159.3190,
14,Itarema,BR,88.65,scattered clouds,-2.9248,-39.9167,
22,Hilo,US,75.27,overcast clouds,19.7297,-155.0900,
23,Bandarbeyla,SO,80.01,light rain,9.4942,50.8122,
24,Alice Town,BS,82.89,scattered clouds,25.7167,-79.3000,
25,Kununurra,AU,76.98,broken clouds,-15.7667,128.7333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Castro,BR,76.48,overcast clouds,-24.7911,-50.0119,CHACARA BAILLY
5,Abu Dhabi,AE,89.71,clear sky,24.4667,54.3667,Ramee Garden Hotel Apartments
6,Maragogi,BR,80.56,overcast clouds,-9.0122,-35.2225,Salinas do Maragogi All Inclusive Resort
8,Aljezur,PT,75.76,broken clouds,37.3191,-8.8033,Amazigh Hostel
10,Kapaa,US,76.19,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
14,Itarema,BR,88.65,scattered clouds,-2.9248,-39.9167,"Sea Lounge! Bar, Club & Restaurant"
22,Hilo,US,75.27,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
23,Bandarbeyla,SO,80.01,light rain,9.4942,50.8122,JABIR HOTEL
24,Alice Town,BS,82.89,scattered clouds,25.7167,-79.3000,Bimini Big Game Club Resort & Marina
25,Kununurra,AU,76.98,broken clouds,-15.7667,128.7333,Hotel Kununurra


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   215
Country                215
Max Temp               215
Current Description    215
Lat                    215
Lng                    215
Hotel Name             215
dtype: int64

In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}<dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}<dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [15]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))